In [1]:
!pip install pykeen -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.3/739.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 11.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 9.2 MB/s eta 0:00:00


In [3]:
import pykeen
# from pykeen.datasets import BioKG
from pykeen.datasets import CoDExSmall
from pykeen.pipeline import pipeline
from pykeen.predict import predict_triples
import torch

In [4]:
dataset = CoDExSmall()

training_data = dataset.training.mapped_triples

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


pipeline_results = pipeline(
    model="TransE",  # TransE model for relation prediction
    dataset=dataset,  # Nations dataset
    training_kwargs=dict(num_epochs=20),  # Number of training epochs
    device=device
)

print(pipeline_results)

INFO:pykeen.pipeline.api:Using device: cuda


Training epochs on cuda:0:   0%|          | 0/20 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0/129 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/129 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/129 [00:00<?, ?batch/s]

...

Training batches on cuda:0:   0%|          | 0/129 [00:00<?, ?batch/s]

INFO:pykeen.evaluation.evaluator:Starting batch_size search for evaluation now...
INFO:pykeen.evaluation.evaluator:Concluded batch_size search with batch_size=2048.


Evaluating on cuda:0:   0%|          | 0.00/1.83k [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 0.24s seconds


PipelineResult(random_seed=2292760764, model=TransE(
  (loss): MarginRankingLoss(
    (margin_activation): ReLU()
  )
  (interaction): TransEInteraction()
  (entity_representations): ModuleList(
    (0): Embedding(
      (_embeddings): Embedding(2034, 50)
    )
  )
  (relation_representations): ModuleList(
    (0): Embedding(
      (_embeddings): Embedding(42, 50)
    )
  )
  (weight_regularizers): ModuleList()
), training=TriplesFactory(num_entities=2034, num_relations=42, create_inverse_triples=False, num_triples=32888, path="/root/.data/pykeen/datasets/codexsmall/train.txt"), training_loop=<pykeen.training.slcwa.SLCWATrainingLoop object at 0x7df5716adcc0>, losses=[0.9956030060154523, 0.7891966409461443, 0.6583014246105223, 0.5591369844222254, 0.47641867006471916, 0.4146629939245623, 0.3630723948626555, 0.3269834002090055, 0.30281054777230404, 0.28222195723260096, 0.2525480619465658, 0.23743758483450542, 0.2210794575223627, 0.21124615976514743, 0.19882621598798175, 0.1844177037015441

In [5]:
pack = predict_triples(model=pipeline_results.model, triples=dataset.validation)
df = pack.process(factory=pipeline_results.training).df

df.nlargest(n=10, columns="score")

,head_id,head_label,relation_id,relation_label,tail_id,tail_label,score
409,466,Q180224,22,P27,1030,Q30,-5.801616
1173,1303,Q41,34,P530,353,Q159,-5.811313
1272,1409,Q45909,7,P1303,1609,Q5994,-5.823368
706,791,Q230395,22,P27,1030,Q30,-5.891012
1448,1673,Q668,34,P530,2008,Q96,-5.927941
1733,1939,Q887347,22,P27,1030,Q30,-5.946043
598,678,Q209471,1,P102,1008,Q29552,-5.965466
505,560,Q189758,22,P27,1030,Q30,-6.022103
1166,1299,Q408,34,P530,1408,Q458,-6.035176
866,977,Q28,34,P530,1927,Q865,-6.041920
